In [1]:
import json
import pandas
#json_file = open("C:\\Users\\abhis\\Desktop\\Project\\greendeck\\News_Category_Dataset1.json",'r')

input_file = open ('News_Category_Dataset.json')
data = json.load(input_file)

category_list = ['POLITICS','ENTERTAINMENT','HEALTHY LIVING','QUEER VOICES','BUSINESS','SPORTS','COMEDY','PARENTS','BLACK VOICES','THE WORLDPOST','WOMEN','CRIME','MEDIA','WEIRD NEWS','GREEN','IMPACT','WORLDPOST','RELIGION','STYLE','WORLD NEWS','TRAVEL','TASTE','ARTS','FIFTY','GOOD NEWS','SCIENCE','ARTS & CULTURE','TECH','COLLEGE','LATINO VOICES','EDUCATION']
training_data = []
for i in range(0,len(data)):
    #print(data[i]['short_description']+' '+data[i]['headline'])
    #print(data[i]['category'])
    cate=data[i]['category']
    #print(category_list.index(cate))
    training_data.append({'data' : data[i]['short_description']+' '+data[i]['headline'], 'flag' : category_list.index(data[i]['category'])})
training_data[0]
#print(training_data)

{'data': 'She left her husband. He killed their children. Just another day in America. There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV',
 'flag': 11}

In [2]:
training_data = pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.size)


124989


In [0]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#for vector count
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)
#print(X_train_counts)
#save vector data
pickle.dump(count_vect.vocabulary_, open("count_vectorizer.pkl","wb"))

#transformation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#print("transform")
#print(X_train_tfidf)
#save data
pickle.dump(tfidf_transformer, open("TfidfTransformer.pkl","wb"))

In [4]:
#softmax
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.20, random_state=42)

clf_neural.fit(X_train, y_train)

pickle.dump(clf_neural, open("softmax.pkl", "wb"))

predicted = clf_neural.predict(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

accuracy = clf_neural.score(X_test, y_test)
print("softmax accuracy",accuracy)

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' -- ', category_list[result])
    


softmax accuracy 0.5402432194575566
POLITICS  --  THE WORLDPOST
POLITICS  --  POLITICS
GREEN  --  SCIENCE
POLITICS  --  BLACK VOICES
BUSINESS  --  POLITICS
ENTERTAINMENT  --  CRIME
POLITICS  --  TRAVEL
POLITICS  --  POLITICS
POLITICS  --  POLITICS
QUEER VOICES  --  QUEER VOICES
GREEN  --  TRAVEL
POLITICS  --  POLITICS
WORLD NEWS  --  THE WORLDPOST
POLITICS  --  POLITICS
RELIGION  --  HEALTHY LIVING
ENTERTAINMENT  --  ENTERTAINMENT
ENTERTAINMENT  --  ENTERTAINMENT
HEALTHY LIVING  --  COLLEGE
POLITICS  --  BUSINESS
GOOD NEWS  --  LATINO VOICES
WOMEN  --  WOMEN
MEDIA  --  POLITICS
BLACK VOICES  --  ENTERTAINMENT
ENTERTAINMENT  --  POLITICS
THE WORLDPOST  --  IMPACT
POLITICS  --  POLITICS
TASTE  --  ARTS
POLITICS  --  MEDIA
POLITICS  --  POLITICS
HEALTHY LIVING  --  RELIGION
POLITICS  --  POLITICS
HEALTHY LIVING  --  WORLDPOST
ENTERTAINMENT  --  ENTERTAINMENT
TASTE  --  HEALTHY LIVING
HEALTHY LIVING  --  HEALTHY LIVING
WOMEN  --  ENTERTAINMENT
POLITICS  --  POLITICS
HEALTHY LIVING  --  HEA

In [5]:
from sklearn.metrics import classification_report
print('classification report of softmax')
print(classification_report(y_test, predicted, target_names=category_list))

classification report of softmax
                precision    recall  f1-score   support

      POLITICS       0.70      0.82      0.76      6522
 ENTERTAINMENT       0.58      0.77      0.66      2905
HEALTHY LIVING       0.46      0.67      0.54      1340
  QUEER VOICES       0.70      0.69      0.70      1002
      BUSINESS       0.43      0.52      0.47       894
        SPORTS       0.62      0.63      0.63       787
        COMEDY       0.41      0.22      0.29       810
       PARENTS       0.47      0.50      0.48       772
  BLACK VOICES       0.36      0.40      0.38       799
 THE WORLDPOST       0.38      0.58      0.46       748
         WOMEN       0.34      0.36      0.35       709
         CRIME       0.50      0.46      0.47       546
         MEDIA       0.56      0.41      0.48       574
    WEIRD NEWS       0.25      0.23      0.24       497
         GREEN       0.34      0.34      0.34       537
        IMPACT       0.18      0.15      0.16       490
     WORLDPOST

In [6]:
#multinomial naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.20, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

pickle.dump(clf, open("mnbmodel.pkl", "wb"))

loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vectorizer.pkl", "rb")))
loaded_tfidf = pickle.load(open("TfidfTransformer.pkl","rb"))
loaded_model = pickle.load(open("mnbmodel.pkl","rb"))

predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_mnbayes.csv', sep = ',')

accuracy = loaded_model.score(X_test, y_test)
print("multinomial naive bayes accuracy",accuracy)

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' -- ', category_list[result])
    


multinomial naive bayes accuracy 0.3587486998959917
POLITICS  --  THE WORLDPOST
POLITICS  --  POLITICS
POLITICS  --  SCIENCE
POLITICS  --  BLACK VOICES
POLITICS  --  POLITICS
ENTERTAINMENT  --  CRIME
POLITICS  --  TRAVEL
POLITICS  --  POLITICS
POLITICS  --  POLITICS
POLITICS  --  QUEER VOICES
POLITICS  --  TRAVEL
POLITICS  --  POLITICS
POLITICS  --  THE WORLDPOST
POLITICS  --  POLITICS
POLITICS  --  HEALTHY LIVING
ENTERTAINMENT  --  ENTERTAINMENT
ENTERTAINMENT  --  ENTERTAINMENT
POLITICS  --  COLLEGE
POLITICS  --  BUSINESS
POLITICS  --  LATINO VOICES
POLITICS  --  WOMEN
POLITICS  --  POLITICS
POLITICS  --  ENTERTAINMENT
POLITICS  --  POLITICS
POLITICS  --  IMPACT
POLITICS  --  POLITICS
POLITICS  --  ARTS
POLITICS  --  MEDIA
POLITICS  --  POLITICS
POLITICS  --  RELIGION
POLITICS  --  POLITICS
POLITICS  --  WORLDPOST
ENTERTAINMENT  --  ENTERTAINMENT
HEALTHY LIVING  --  HEALTHY LIVING
POLITICS  --  HEALTHY LIVING
POLITICS  --  ENTERTAINMENT
POLITICS  --  POLITICS
POLITICS  --  HEALTHY LIV

In [7]:
from sklearn.metrics import classification_report
print('classification report of multinomial naive bayes')
print(classification_report(y_test, predicted, target_names=category_list))

classification report of multinomial naive bayes
                precision    recall  f1-score   support

      POLITICS       0.32      1.00      0.48      6522
 ENTERTAINMENT       0.51      0.71      0.59      2905
HEALTHY LIVING       0.72      0.16      0.26      1340
  QUEER VOICES       0.97      0.04      0.07      1002
      BUSINESS       0.80      0.00      0.01       894
        SPORTS       0.95      0.05      0.10       787
        COMEDY       0.00      0.00      0.00       810
       PARENTS       1.00      0.03      0.07       772
  BLACK VOICES       0.00      0.00      0.00       799
 THE WORLDPOST       0.48      0.01      0.03       748
         WOMEN       1.00      0.02      0.05       709
         CRIME       0.67      0.01      0.01       546
         MEDIA       0.00      0.00      0.00       574
    WEIRD NEWS       0.00      0.00      0.00       497
         GREEN       1.00      0.00      0.01       537
        IMPACT       0.00      0.00      0.00       49

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
#Support vector machine
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.20, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')


accuracy = clf_svm.score(X_test, y_test)
print("svm accuracy",accuracy)

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' -- ', category_list[result])


svm accuracy 0.9196735738859109
POLITICS  --  THE WORLDPOST
POLITICS  --  POLITICS
SCIENCE  --  SCIENCE
BLACK VOICES  --  BLACK VOICES
POLITICS  --  POLITICS
ENTERTAINMENT  --  CRIME
TRAVEL  --  TRAVEL
POLITICS  --  POLITICS
POLITICS  --  POLITICS
QUEER VOICES  --  QUEER VOICES
TRAVEL  --  TRAVEL
POLITICS  --  POLITICS
THE WORLDPOST  --  THE WORLDPOST
POLITICS  --  POLITICS
HEALTHY LIVING  --  HEALTHY LIVING
ENTERTAINMENT  --  ENTERTAINMENT
ENTERTAINMENT  --  ENTERTAINMENT
COLLEGE  --  COLLEGE
BUSINESS  --  BUSINESS
LATINO VOICES  --  LATINO VOICES
WOMEN  --  WOMEN
MEDIA  --  POLITICS
BLACK VOICES  --  ENTERTAINMENT
POLITICS  --  POLITICS
IMPACT  --  IMPACT
POLITICS  --  POLITICS
ARTS  --  ARTS
MEDIA  --  MEDIA
POLITICS  --  POLITICS
RELIGION  --  RELIGION
POLITICS  --  POLITICS
WORLDPOST  --  WORLDPOST
ENTERTAINMENT  --  ENTERTAINMENT
HEALTHY LIVING  --  HEALTHY LIVING
HEALTHY LIVING  --  HEALTHY LIVING
ENTERTAINMENT  --  ENTERTAINMENT
POLITICS  --  POLITICS
HEALTHY LIVING  --  HEALTH

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print('classification report of Support vector machine')
print(classification_report(y_test, predicted, target_names=category_list))
print(accuracy_score(y_test, predicted, normalize=False))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predicted))


classification report of Support vector machine
                precision    recall  f1-score   support

      POLITICS       0.90      0.96      0.93      6522
 ENTERTAINMENT       0.91      0.95      0.93      2905
HEALTHY LIVING       0.87      0.92      0.89      1340
  QUEER VOICES       0.95      0.93      0.94      1002
      BUSINESS       0.93      0.87      0.90       894
        SPORTS       0.92      0.96      0.94       787
        COMEDY       0.93      0.81      0.86       810
       PARENTS       0.89      0.92      0.90       772
  BLACK VOICES       0.92      0.86      0.89       799
 THE WORLDPOST       0.92      0.91      0.92       748
         WOMEN       0.91      0.82      0.86       709
         CRIME       0.94      0.93      0.93       546
         MEDIA       0.93      0.85      0.89       574
    WEIRD NEWS       0.97      0.93      0.95       497
         GREEN       0.91      0.92      0.92       537
        IMPACT       0.94      0.83      0.88       490

In [0]:
#-Abhishek Shingadiya (abhishekshingadiya2543@gmail.com,7046685875,7048896523)